In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import pandas as pd
from finbert_embedding.embedding import FinbertEmbedding
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
from tqdm import tqdm
import glob
import os.path
from pandas import *
from collections import Counter
import time

In [2]:
current_data_directory = os.path.abspath("./data/sp500")
path_text = current_data_directory + "/text/"
path_to_id_csv = current_data_directory + "/df_100stocks_sp500.csv"


In [3]:
print(current_data_directory)

/Users/juliencyrusenyegue/Documents/GitHub/financial_econometrics_project/FinBert/data/sp500


In [4]:
data = read_csv(path_to_id_csv)
ID_sp500 = data['id'].tolist()

In [5]:
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

In [6]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

In [7]:
# Extract texts from the directory

def extract_texts():

    texts = []
    first_sentence = []
    articles = []
    list_ID = []
   
    list_tickers = os.listdir(path_text)
    #list_tickers.remove('.DS_Store') # for mac
    
    for ticker in list_tickers:
        
        list_articles = glob.glob(path_text+ticker+"/*")
        
        ID = []
        for s in list_articles:
            
            test_id = int(s.replace(path_text+str(ticker)+"/", ""))
            
            
            if test_id in ID_sp500:
                
                with open(s) as f:
                    
                    x = re.sub(path_text+str(ticker)+"/","",s)
                    articles.append(x)
                    t = f.read()
                    texts.append(t)
        
                ID.append(test_id)
                  
        list_ID += ID
    
    print(len(texts))
     
    return texts, list_ID


In [8]:
def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]

    #sentences = [x for x in sentences if len(x.split(' '))>5]
    return sentences


In [9]:
# Run finbert on each text and count the nb of neutral, positive, negative sentences

def finbert_texts(start,n_texts, texts):
    labels = {0:'neutral', 1:'positive',2:'negative'}
    data = np.zeros((n_texts,3))
    
    for j in tqdm(range(start,start + n_texts)):
        
        print(j - start + 1)
        #sentences = split_into_sentences(texts[j])
        sentences = texts[j].split('\n \n')
        sentences = [x for x in sentences if len(x.split(' '))>4]
        inputs = tokenizer(sentences, return_tensors="pt", padding=True,truncation=True,max_length=512)


        outputs = finbert(**inputs)[0]
        #print(outputs)
        i = np.argmax(outputs.detach().numpy(), axis = 1)
        #print(i)
        sentiment_counter = list(Counter(i).values())
        if(np.shape(sentiment_counter) == (2, )):
            sentiment_counter.append(0)
        print(sentiment_counter)
        data[j-start,:] = sentiment_counter
        
        time.sleep(45)
        
    return data

In [10]:
# Nombre d'articles
N = 98
start = 302

li = [1,2]
#print(np.shape(li))

if(np.shape(li) == (2, )):
    li.append(4)
    print(li)
    

In [ ]:
 if __name__ == "__main__":
    
    texts, list_ID = extract_texts()
    data = finbert_texts(start, N, texts)
    
   

1300


  0%|                                                    | 0/98 [00:00<?, ?it/s]

1
[36, 40, 3]


  1%|▍                                         | 1/98 [01:13<1:58:34, 73.34s/it]

2
[58, 25, 3]


  2%|▊                                         | 2/98 [03:10<2:38:47, 99.25s/it]

3
[54, 29, 6]


  3%|█▎                                       | 3/98 [05:18<2:57:40, 112.21s/it]

4


In [ ]:
# Ratios computations

df_data = pd.DataFrame(data, columns= ["Neutral","Positive", "Negative"])


df_data["Total"] = df_data["Neutral"] + df_data["Positive"] + df_data["Negative"]
df_data["Difference"] = df_data["Positive"] - df_data["Negative"]
df_data["P_N ratio"] = df_data["Positive"] / df_data["Negative"]
df_data["Difference ratio"] = df_data["Difference"] / df_data["Total"]

df_data["Decile"] = 1 + df_data["Difference ratio"].transform(lambda y: pd.qcut(y, 10, labels=False))

df_data = np.round(df_data, 5)


df_data["ID"] = list_ID[start: start + N]
first_column = df_data.pop("ID")
df_data.insert(0, "ID", first_column)

#long = list(df_data.loc[df_data['Decile'].isin([10,9])]["ID"])
#short = list(df_data.loc[df_data['Decile'].isin([1,2])]["ID"])

display(df_data)


In [ ]:
# Pas run à la première itération

df_ratios = read_csv(os.path.abspath("./FinBert_ratios.csv"))
display(df_ratios)
df_ratios = pd.concat([df_ratios, df_data])
df_ratios["Decile"] = 1 + df_ratios["Difference ratio"].transform(lambda y: pd.qcut(y, 10, labels=False))
display(df_ratios)

In [ ]:
df_ratios.to_csv('FinBert_ratios.csv', index = False)